In [2]:
# girdsearchCV 최적의 하이퍼 파라미터 도출
# ex)max_depth, min_samples_split


In [4]:
# cross_val_scord()

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy), 교차 검증 세트는 3개
scores = cross_val_score(dt_clf, data, label, scoring = 'accuracy' , cv = 3)
print('교차 검증별 정확도: ', np.round(scores ,4 ))
print('평균 검증 정확도 ;' , np.round(np.mean(scores), 4))

교차 검증별 정확도:  [0.98 0.94 0.98]
평균 검증 정확도 ; 0.9667


In [11]:
# GridsearchCV

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# 데이터를 로딩하고 학습데이터와 테스트데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data , iris_data.target, test_size = 0.2, random_state=121)
dtree = DecisionTreeClassifier()

### parameter 들을 dictionary 형태로 설정
parameters = {'max_depth': [1,2,3], 'min_samples_split':[2,3]}

In [13]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train,test set fold 로 나누어서 테스트 수행 설정
### refit = True 가 default 임. True이면 가장 좋은 파라미터 설정으로 재합습 시킴.
grid_dtree = GridSearchCV(dtree, param_grid= parameters, cv =3, refit = True, return_train_score = True)

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습.평가
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과는 cv_results_ 라는 딕셔너리로 저장됨. 이를 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [14]:
scores_df.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_max_depth', 'param_min_samples_split', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score',
       'split0_train_score', 'split1_train_score', 'split2_train_score',
       'mean_train_score', 'std_train_score'],
      dtype='object')

In [15]:
grid_dtree.cv_results_

{'mean_fit_time': array([0.00066495, 0.00033267, 0.00032989, 0.00043305, 0.00064079,
        0.00067822]),
 'std_fit_time': array([0.00047019, 0.00047047, 0.00046654, 0.0004109 , 0.00045562,
        0.00047976]),
 'mean_score_time': array([0.00066535, 0.00034388, 0.        , 0.        , 0.        ,
        0.00033124]),
 'std_score_time': array([0.00047047, 0.00048632, 0.        , 0.        , 0.        ,
        0.00046845]),
 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1, 'min_samples_split': 2},
  {'max_depth': 1, 'min_samples_split': 3},
  {'max_depth': 2, 'min_samples_split': 2},
  {'max_depth': 2, 'min_samples_split': 3},
  {'max_depth': 3, 'min_sample

In [16]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSerachCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

#refit = Treu 로 설정된 GridSearchCV 객체가 fit()을 수행 시 학습이 완료된 esimator를 내포하고 있으므로 predict()를 통해 예측가능
pred = grid_dtree.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSerachCV 최고 정확도: 0.9750
테스트 데이터 세트 정확도: 0.9667


In [19]:
# GridSearchCV의 refit으로 이미학습이 된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도 : {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도 : 0.9667


In [ ]:
릿지 랏소